<a href="https://colab.research.google.com/github/Onyimatics/msc_dissertation/blob/master/second_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install country_converter


In [ ]:
import pandas as pd
import numpy as np
import re
import os


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
file_path = '/content/drive/MyDrive/Colab Notebooks/BCU/MSc Dissertation/fibromyalgia_forum.csv'
df = pd.read_csv(file_path)

In [ ]:
print(df.shape)
df.head()

(67139, 23)


,thread_id,thread_title,thread_url,forum_category,thread_views,thread_replies,total_posts,post_type,post_index,post_id,...,content,content_length,user_joined,user_messages,user_country,user_state,has_quotes,quote_count,all_quotes,quoted_users
0,my-advice-for-managing-fibromyalgia-especially...,My advice for managing fibromyalgia (especiall...,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,27K,29,30,original_post,0,post-302771,...,"I am not a doctor or anything, just a person w...",10478,"Dec 2, 2016","4,081",US,NaN,No,0,NaN,NaN
1,my-advice-for-managing-fibromyalgia-especially...,My advice for managing fibromyalgia (especiall...,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,27K,29,30,reply,1,post-302807,...,"Hi Sunkacola, I am quite new to this forum. I ...",1757,"Jun 25, 2020",74,AU,NaN,No,0,NaN,NaN
2,my-advice-for-managing-fibromyalgia-especially...,My advice for managing fibromyalgia (especiall...,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,27K,29,30,reply,2,post-302809,...,"Hi everyone. This may or may not be of use, ...",942,"Feb 26, 2020",6,US,MO,No,0,NaN,NaN
3,my-advice-for-managing-fibromyalgia-especially...,My advice for managing fibromyalgia (especiall...,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,27K,29,30,reply,3,post-302813,...,"Hello Eff, Thanks very much for your lovely co...",781,"Dec 2, 2016","4,081",US,NaN,No,0,NaN,NaN
4,my-advice-for-managing-fibromyalgia-especially...,My advice for managing fibromyalgia (especiall...,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,27K,29,30,reply,4,post-302814,...,"Hi DizzyLiz, I have never heard of PPPD, so th...",1274,"Dec 2, 2016","4,081",US,NaN,No,0,NaN,NaN


In [ ]:
# Get a summary of columns, data types, and null counts
df.info()

# Get the number of unique values per column
print(df.nunique())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67139 entries, 0 to 67138
Data columns (total 23 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   thread_id       67139 non-null  object
 1   thread_title    67139 non-null  object
 2   thread_url      67139 non-null  object
 3   forum_category  67139 non-null  object
 4   thread_views    67139 non-null  object
 5   thread_replies  67139 non-null  int64 
 6   total_posts     67139 non-null  int64 
 7   post_type       67139 non-null  object
 8   post_index      67139 non-null  int64 
 9   post_id         67139 non-null  object
 10  username        67139 non-null  object
 11  user_title      67139 non-null  object
 12  timestamp       67139 non-null  object
 13  content         67095 non-null  object
 14  content_length  67139 non-null  int64 
 15  user_joined     65732 non-null  object
 16  user_messages   65732 non-null  object
 17  user_country    62769 non-null  object
 18  user_s

In [ ]:
# Count missing values for each column
missing = df.isnull().sum()
print("Missing values per column:\n", missing)

# Percentage of missing values
missing_percent = (df.isnull().sum() / len(df)) * 100
print("Percentage missing per column:\n", missing_percent)


Missing values per column:
 thread_id             0
thread_title          0
thread_url            0
forum_category        0
thread_views          0
thread_replies        0
total_posts           0
post_type             0
post_index            0
post_id               0
username              0
user_title            0
timestamp             0
content              44
content_length        0
user_joined        1407
user_messages      1407
user_country       4370
user_state        18358
has_quotes            0
quote_count           0
all_quotes        56577
quoted_users      56577
dtype: int64
Percentage missing per column:
 thread_id          0.000000
thread_title       0.000000
thread_url         0.000000
forum_category     0.000000
thread_views       0.000000
thread_replies     0.000000
total_posts        0.000000
post_type          0.000000
post_index         0.000000
post_id            0.000000
username           0.000000
user_title         0.000000
timestamp          0.000000
content    

In [ ]:
# Check for duplicate post_ids and thread_ids
print("Duplicate post_id count:", df['post_id'].duplicated().sum())
print("Duplicate thread_id count:", df['thread_id'].duplicated().sum())

# Check for exact duplicate rows
print("Exact duplicate rows:", df.duplicated().sum())


Duplicate post_id count: 8
Duplicate thread_id count: 58785
Exact duplicate rows: 0


In [ ]:
# Check if all posts are mapped to threads
print("Posts without thread_id:", df['thread_id'].isnull().sum())

# Check post_index range within threads
post_index_stats = df.groupby('thread_id')['post_index'].agg(['min', 'max', 'count'])
print(post_index_stats.head())

# Check if post_index starts at 0 for each thread
print("Threads where post_index does not start at 0:",
      (post_index_stats['min'] != 0).sum())


Posts without thread_id: 0
                                           min  max  count
thread_id                                                 
1-beats-0-red-heart.30812                    0    9     10
1-fibromyalgia-2-gambling.23826              0    5      6
15-years-since-diagnosis.25835               0    1      2
18-and-chronically-ill.28720                 0    5      6
18-years-of-pain-recently-diagnosed.28507    0   29     30
Threads where post_index does not start at 0: 0


In [ ]:
# Check for missing or malformed timestamps
print("Missing timestamps:", df['timestamp'].isnull().sum())

# Try parsing timestamps to datetime
df['timestamp_parsed'] = pd.to_datetime(df['timestamp'], errors='coerce')
print("Malformed timestamps:", df['timestamp_parsed'].isnull().sum())


Missing timestamps: 0
Malformed timestamps: 0


/tmp/ipython-input-10-417558227.py:5: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df['timestamp_parsed'] = pd.to_datetime(df['timestamp'], errors='coerce')


In [ ]:
# Check for missing or inconsistent user metadata
user_metadata = ['username', 'user_title', 'user_joined', 'user_country', 'user_state']
for col in user_metadata:
    print(f"{col}: missing={df[col].isnull().sum()}, unique={df[col].nunique()}")


username: missing=0, unique=5706
user_title: missing=0, unique=11
user_joined: missing=1407, unique=2744
user_country: missing=4370, unique=159
user_state: missing=18358, unique=769


In [ ]:
# Check missing content and content length
print("Missing content:", df['content'].isnull().sum())
print("Content length stats:\n", df['content_length'].describe())


Missing content: 44
Content length stats:
 count    67139.000000
mean       705.426503
std        815.561305
min          0.000000
25%        229.000000
50%        471.000000
75%        890.000000
max      47987.000000
Name: content_length, dtype: float64


In [ ]:
# Example: Are missing user_country values concentrated among certain users?
missing_country_users = df[df['user_country'].isnull()]['username'].value_counts().head()
print("Users with most missing user_country:\n", missing_country_users)

# Example: Are missing content values concentrated in certain threads?
missing_content_threads = df[df['content'].isnull()]['thread_id'].value_counts().head()
print("Threads with most missing content:\n", missing_content_threads)


Users with most missing user_country:
 username
Guest                            1339
SweetWithSour                     140
Momzilla                           54
User Requested Deletion 36696      53
thparker13                         46
Name: count, dtype: int64
Threads with most missing content:
 thread_id
sparse-eye-lashes-and-brows.29764                  2
post-surgery.29797                                 2
how-are-you-today.29484                            2
fibromyalgia-new-insights-new-hope-part-1.27559    2
vivid-dreams.30401                                 1
Name: count, dtype: int64


In [ ]:
# Show duplicate post_ids and their rows
duplicate_post_ids = df[df['post_id'].duplicated(keep=False)]
print(duplicate_post_ids[['post_id', 'thread_id', 'username', 'timestamp', 'content']])


           post_id                                          thread_id  \
5554   post-313494  burning-feet-foot-can-i-ask-if-anyone-suffers-...   
5569   post-313494  burning-feet-foot-can-i-ask-if-anyone-suffers-...   
5895   post-311491                                          ldn.29050   
5910   post-311491                                          ldn.29050   
6286   post-311286              sudden-muscle-pain-and-weakness.29017   
6301   post-311286              sudden-muscle-pain-and-weakness.29017   
7060   post-309446   any-idea-about-how-to-fight-with-dizziness.28852   
7075   post-309446   any-idea-about-how-to-fight-with-dizziness.28852   
7497   post-308161                         how-do-i-lose-weight.28751   
7512   post-308161                         how-do-i-lose-weight.28751   
28464  post-314685  how-to-ask-my-doctor-can-i-chat-with-one-of-yo...   
28479  post-314685  how-to-ask-my-doctor-can-i-chat-with-one-of-yo...   
32388  post-315093  recently-diagnosed-looking-for-

In [ ]:
# Users with most missing user_state
missing_state_users = df[df['user_state'].isnull()]['username'].value_counts().head(10)
print("Users with most missing user_state:\n", missing_state_users)

# Forum categories with most missing user_country
missing_country_categories = df[df['user_country'].isnull()]['forum_category'].value_counts()
print("Forum categories with most missing user_country:\n", missing_country_categories)


Users with most missing user_state:
 username
sunkacola       4029
JayCS           3134
Guest           1339
Jemima           826
SBee             714
Badger           622
sweetkamie20     470
hope23           254
fimi             248
TipBill          227
Name: count, dtype: int64
Forum categories with most missing user_country:
 forum_category
General discussions about fibromyalgia    1739
Do I have fibromyalgia?                    528
Chronic pain                               421
Chronic fatigue                            418
Welcome new members                        373
Recently diagnosed                         231
Moan, complain and vent                    226
Help and inspiration                       131
Alternative medicines and therapies        106
Hobbies and interests                       52
Tea time and pub                            42
Fibro fog                                   38
Pharmaceuticals and pharmacology            31
Research and news                         

In [ ]:
# Show the top 5 longest posts
longest_posts = df.sort_values('content_length', ascending=False).head(5)
print(longest_posts[['post_id', 'username', 'content_length', 'content']])


           post_id   username  content_length  \
26990  post-211963      Guest           47987   
53119  post-289931  gryfalcon           19589   
57061  post-319032      JayCS           16354   
6089   post-309427      JayCS           15231   
6088   post-309426      JayCS           15054   

                                                 content  
26990  Protomyxozoa Rheumatica is a parasite discover...  
53119  Full disclosure.  I have no financial ties to ...  
57061  So many brilliant ideas! In some cases I'd lik...  
6089   This is the "spoilered" version of my long pos...  
6088   See also the "spoilered"/structured version .....  


In [ ]:
# Unique country/state values (sample)
print("Sample user_country values:", df['user_country'].dropna().unique()[:20])
print("Sample user_state values:", df['user_state'].dropna().unique()[:20])


Sample user_country values: ['US' 'AU' 'DE' 'CA' 'UK' 'RO' 'CY' 'NZ' 'RF' 'IE' 'SA' 'ES' 'WS' 'EU'
 'DK' 'GR' 'HU' 'IN' 'SE' 'PT']
Sample user_state values: ['MO' 'TX' 'IN' 'BC' 'CO' 'WI' 'CA' 'Canada' 'NJ' 'LA' 'VI' 'GA' 'NS' 'OH'
 'WA' 'FL' 'NI' 'IL' 'VA' 'NC']


In [ ]:
# Find threads where post_index is not unique
duplicates_within_thread = df.groupby('thread_id')['post_index'].nunique() < df.groupby('thread_id')['post_index'].count()
print("Threads with duplicate post_index values:", duplicates_within_thread.sum())


Threads with duplicate post_index values: 0


In [ ]:
guest_deleted = df[df['username'].str.contains('Guest|User Requested Deletion', na=False)]
print("Number of posts from Guest or deleted users:", len(guest_deleted))
print("Percentage of dataset:", len(guest_deleted) / len(df) * 100)


Number of posts from Guest or deleted users: 1392
Percentage of dataset: 2.0733105944384036


In [ ]:
import pandas as pd
import numpy as np
import country_converter as coco

# Assuming your DataFrame is named df and has a 'user_country' column

cc = coco.CountryConverter()

# Convert country codes to country names, set unconverted to 'not found', then replace with NaN
df['country_name'] = cc.convert(names=df['user_country'].tolist(), to='name_short', not_found='not found')
df['country_name'] = df['country_name'].replace('not found', np.nan)

# Count successful and failed conversions
success_count = df['country_name'].notnull().sum()
fail_count = df['country_name'].isnull().sum()

print(f"Successfully converted: {success_count}")
print(f"Failed to convert: {fail_count}")

# Show user_country codes that failed to convert (for manual review)
failed_codes = df[df['country_name'].isnull()]['user_country'].unique()
print("Failed codes:", failed_codes)

Streaming output truncated to the last 5000 lines.


Successfully converted: 61522
Failed to convert: 5617
Failed codes: [nan 'RF' 'EU' 'AN' 'UR' 'Ire' 'Sve' 'YS' 'Mor' 'KYA' 'Mal' 'TA' 'SUD'
 'USp' 'MI' 'IRE' 'dil' 'OH' 'Rep' 'UAE' 'Por' 'RSA' 'CT' 'Rom' 'Cou'
 'NED' 'AXA' 'VIC' 'UU' 'SP' 'MRU' 'FL' 'AUA' 'Net' 'Ser' 'GER' 'Sin'
 'CRO' 'PHI' 'BNG' 'QR' 'Lh' 'JA' 'USE' 'USI' 'NIG' 'Vic' 'Ger' 'Php'
 'GAM' 'WA' 'Phi' 'ENG' 'SLO' 'BAH' 'Mon']


In [ ]:
# manual_country_map = {
#     'KYA': 'Kenya',
#     'NIG': 'Nigeria',
#     'Mal': 'Malaysia',   # or 'Malta' if context fits
#     'TA': None,          # Not enough info, leave as None
#     'SUD': 'Sudan',
#     'IRE': 'Ireland',
#     'Rep': None,         # Could be 'Republic of...' but ambiguous
#     'Vic': None,         # Could be a region, not a country
#     'Sin': 'Singapore',
#     'GAM': 'Gambia',
#     'SLP': None,         # Not enough info
#     'Php': 'Philippines',
#     # Add more mappings as you review your data
# }

In [ ]:
# Number of successful conversions (country_name not null)
success_count = df['country_name'].notnull().sum()

# Number of failed conversions (country_name is null)
fail_count = df['country_name'].isnull().sum()

print(f"Successfully converted: {success_count}")
print(f"Failed to convert: {fail_count}")


Successfully converted: 61522
Failed to convert: 5617


In [ ]:
# Show user_country codes that failed to convert (for manual review)
failed_codes = df[df['country_name'].isnull()]['user_country'].unique()
print("Failed codes:", failed_codes)


Failed codes: [nan 'RF' 'EU' 'AN' 'UR' 'Ire' 'Sve' 'YS' 'Mor' 'KYA' 'Mal' 'TA' 'SUD'
 'USp' 'MI' 'IRE' 'dil' 'OH' 'Rep' 'UAE' 'Por' 'RSA' 'CT' 'Rom' 'Cou'
 'NED' 'AXA' 'VIC' 'UU' 'SP' 'MRU' 'FL' 'AUA' 'Net' 'Ser' 'GER' 'Sin'
 'CRO' 'PHI' 'BNG' 'QR' 'Lh' 'JA' 'USE' 'USI' 'NIG' 'Vic' 'Ger' 'Php'
 'GAM' 'WA' 'Phi' 'ENG' 'SLO' 'BAH' 'Mon']


In [ ]:
df.country_name.unique()

array(['United States', 'Australia', 'Germany', 'Canada',
       'United Kingdom', 'Romania', 'Cyprus', nan, 'New Zealand',
       'Ireland', 'Saudi Arabia', 'Spain', 'Samoa', 'Denmark', 'Greece',
       'Hungary', 'India', 'Sweden', 'Portugal', 'Iran', 'Finland',
       'Netherlands', 'Timor-Leste', 'France', 'Albania', 'South Africa',
       'Ukraine', 'Nicaragua', 'Indonesia', 'Peru', 'Colombia', 'Mexico',
       'Norway', 'Japan', 'South Korea', 'Philippines', 'Guam',
       'Malaysia', 'Macau', 'Slovenia', 'Iceland', 'Russia', 'Italy',
       'American Samoa', 'Pakistan', 'Morocco', 'Singapore',
       'Switzerland', 'Georgia', 'Taiwan', 'Thailand', 'Kenya', 'Bhutan',
       'Dominican Republic', 'Türkiye', 'Argentina', 'Czechia',
       'New Caledonia', 'North Macedonia', 'Belgium', 'Vietnam', 'Sudan',
       'El Salvador', 'Ecuador', 'Seychelles', 'Jamaica', 'Egypt',
       'Poland', 'Bangladesh'], dtype=object)

In [ ]:
# Filter the DataFrame to find all rows where 'user_country' is 'KYA'
kya_df = df[df['user_country'] == 'Vic']

# Display the first few rows of the filtered data
print(f"Found {len(kya_df)} rows where user_country is 'KYA'.")
display(kya_df.head())

Found 1 rows where user_country is 'KYA'.


,thread_id,thread_title,thread_url,forum_category,thread_views,thread_replies,total_posts,post_type,post_index,post_id,...,user_joined,user_messages,user_country,user_state,has_quotes,quote_count,all_quotes,quoted_users,timestamp_parsed,country_name
51263,what-do-you-use-for-fibro-fog.26147,What Do You Use for FIBRO FOG?,https://www.fibromyalgiaforums.org/community/t...,Fibro fog,16K,34,35,reply,15,post-293370,...,"Jul 3, 2016",1,Vic,Victoria,No,0,NaN,NaN,2016-08-06 04:19:51-04:00,NaN


In [ ]:
# Assume 'kya_df' is the DataFrame you created in the previous step,
# containing only the 267 rows where user_country is 'KYA'.

# Count the number of unique usernames in this filtered DataFrame
num_unique_users = kya_df['username'].nunique()

print(f"Analysis of the 267 rows where user_country is 'KYA':")
print("----------------------------------------------------")
print(f"These rows belong to {num_unique_users} unique user(s).")

# To see exactly which user(s) and their post counts:
if num_unique_users > 0:
    print("\nBreakdown by user:")
    display(kya_df['username'].value_counts())

Analysis of the 267 rows where user_country is 'KYA':
----------------------------------------------------
These rows belong to 1 unique user(s).

Breakdown by user:


,count
username,
Melbourne,1


In [ ]:
# Assume 'df' is your main DataFrame that has all the data

# Define the list of the two users you want to inspect
# users_to_check = ['vickythecat', 'Nosso88']
users_to_check = ['JDJ']

# Filter the DataFrame to get only the rows belonging to these two users
filtered_users_df = df[df['username'].isin(users_to_check)]

# Select and display only the username, user_country, and user_state columns
print("Displaying user_country and user_state for the identified users:")
display(filtered_users_df[['username', 'user_country', 'user_state', 'user_joined']])

Displaying user_country and user_state for the identified users:


,username,user_country,user_state,user_joined
8852,JDJ,AN,BU,"Feb 7, 2021"


In [ ]:
# Filter the DataFrame to show rows where 'user_country' is null
null_user_country_rows = df[df['user_country'].isnull()]

print("Rows where user_country is null:")
display(null_user_country_rows)

Rows where user_country is null:


,thread_id,thread_title,thread_url,forum_category,thread_views,thread_replies,total_posts,post_type,post_index,post_id,...,user_joined,user_messages,user_country,user_state,has_quotes,quote_count,all_quotes,quoted_users,timestamp_parsed,country_name
64,tremor-and-loss-of-hand-dexterity-and-strength...,Tremor and loss of hand dexterity and strength,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,501,36,37,reply,8,post-329298,...,NaN,NaN,NaN,NaN,No,0,NaN,NaN,2025-05-11 10:46:33-04:00,NaN
66,tremor-and-loss-of-hand-dexterity-and-strength...,Tremor and loss of hand dexterity and strength,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,501,36,37,reply,10,post-329307,...,NaN,NaN,NaN,NaN,No,0,NaN,NaN,2025-05-11 16:17:02-04:00,NaN
69,tremor-and-loss-of-hand-dexterity-and-strength...,Tremor and loss of hand dexterity and strength,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,501,36,37,reply,13,post-329316,...,"Apr 23, 2025",4,NaN,NaN,No,0,NaN,NaN,2025-05-12 11:01:32-04:00,NaN
70,tremor-and-loss-of-hand-dexterity-and-strength...,Tremor and loss of hand dexterity and strength,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,501,36,37,reply,14,post-329338,...,"May 9, 2025",4,NaN,NaN,No,0,NaN,NaN,2025-05-13 07:52:47-04:00,NaN
71,tremor-and-loss-of-hand-dexterity-and-strength...,Tremor and loss of hand dexterity and strength,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,501,36,37,reply,15,post-329339,...,"May 9, 2025",4,NaN,NaN,No,0,NaN,NaN,2025-05-13 07:54:04-04:00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66630,biking.28701,Biking!,https://www.fibromyalgiaforums.org/community/t...,Hobbies and interests,2K,27,28,reply,9,post-308023,...,NaN,NaN,NaN,NaN,No,0,NaN,NaN,2021-08-23 03:36:31-04:00,NaN
66635,biking.28701,Biking!,https://www.fibromyalgiaforums.org/community/t...,Hobbies and interests,2K,27,28,reply,14,post-308096,...,NaN,NaN,NaN,NaN,No,0,NaN,NaN,2021-08-24 13:54:51-04:00,NaN
66640,biking.28701,Biking!,https://www.fibromyalgiaforums.org/community/t...,Hobbies and interests,2K,27,28,reply,19,post-308141,...,NaN,NaN,NaN,NaN,Yes,2,Jemima said:\n\t\t\t\n\t\t\n\t\n\t\n\t\t\n\t\t...,Jemima | Jemima,2021-08-26 14:36:33-04:00,NaN
66644,biking.28701,Biking!,https://www.fibromyalgiaforums.org/community/t...,Hobbies and interests,2K,27,28,reply,23,post-308523,...,NaN,NaN,NaN,NaN,No,0,NaN,NaN,2021-09-14 18:24:47-04:00,NaN


In [ ]:
# Filter the DataFrame to show rows where 'country_name' is null
null_country_name_rows = df[df['country_name'].isnull()]

print("\nRows where country_name is null:")
display(null_country_name_rows)


Rows where country_name is null:


,thread_id,thread_title,thread_url,forum_category,thread_views,thread_replies,total_posts,post_type,post_index,post_id,...,user_joined,user_messages,user_country,user_state,has_quotes,quote_count,all_quotes,quoted_users,timestamp_parsed,country_name
64,tremor-and-loss-of-hand-dexterity-and-strength...,Tremor and loss of hand dexterity and strength,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,501,36,37,reply,8,post-329298,...,NaN,NaN,NaN,NaN,No,0,NaN,NaN,2025-05-11 10:46:33-04:00,NaN
66,tremor-and-loss-of-hand-dexterity-and-strength...,Tremor and loss of hand dexterity and strength,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,501,36,37,reply,10,post-329307,...,NaN,NaN,NaN,NaN,No,0,NaN,NaN,2025-05-11 16:17:02-04:00,NaN
69,tremor-and-loss-of-hand-dexterity-and-strength...,Tremor and loss of hand dexterity and strength,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,501,36,37,reply,13,post-329316,...,"Apr 23, 2025",4,NaN,NaN,No,0,NaN,NaN,2025-05-12 11:01:32-04:00,NaN
70,tremor-and-loss-of-hand-dexterity-and-strength...,Tremor and loss of hand dexterity and strength,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,501,36,37,reply,14,post-329338,...,"May 9, 2025",4,NaN,NaN,No,0,NaN,NaN,2025-05-13 07:52:47-04:00,NaN
71,tremor-and-loss-of-hand-dexterity-and-strength...,Tremor and loss of hand dexterity and strength,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,501,36,37,reply,15,post-329339,...,"May 9, 2025",4,NaN,NaN,No,0,NaN,NaN,2025-05-13 07:54:04-04:00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67103,the-joy-of-bird-watching.21446,The Joy Of Bird Watching,https://www.fibromyalgiaforums.org/community/t...,Hobbies and interests,3K,13,14,reply,5,post-262984,...,"Apr 12, 2013",21,MRU,gport,No,0,NaN,NaN,2013-04-15 13:41:03-04:00,NaN
67106,the-joy-of-bird-watching.21446,The Joy Of Bird Watching,https://www.fibromyalgiaforums.org/community/t...,Hobbies and interests,3K,13,14,reply,8,post-263015,...,"Apr 12, 2013",21,MRU,gport,No,0,NaN,NaN,2013-04-16 17:04:03-04:00,NaN
67121,gardening.21554,Gardening,https://www.fibromyalgiaforums.org/community/t...,Hobbies and interests,3K,6,7,reply,5,post-263360,...,"Apr 23, 2013",26,SP,san cristobal,No,0,NaN,NaN,2013-04-28 05:07:03-04:00,NaN
67128,stargazers.21539,stargazers,https://www.fibromyalgiaforums.org/community/t...,Hobbies and interests,3K,9,10,reply,5,post-262900,...,"Apr 12, 2013",21,MRU,gport,No,0,NaN,NaN,2013-04-12 16:34:57-04:00,NaN


In [ ]:
# Show user_country codes that failed to convert (for manual review)
failed_codes = df[df['country_name'].isnull()]['user_country'].unique()
print("Failed codes:", failed_codes)

Failed codes: [nan 'RF' 'EU' 'AN' 'UR' 'Ire' 'Sve' 'YS' 'Mor' 'KYA' 'Mal' 'TA' 'SUD'
 'USp' 'MI' 'IRE' 'dil' 'OH' 'Rep' 'UAE' 'Por' 'RSA' 'CT' 'Rom' 'Cou'
 'NED' 'AXA' 'VIC' 'UU' 'SP' 'MRU' 'FL' 'AUA' 'Net' 'Ser' 'GER' 'Sin'
 'CRO' 'PHI' 'BNG' 'QR' 'Lh' 'JA' 'USE' 'USI' 'NIG' 'Vic' 'Ger' 'Php'
 'GAM' 'WA' 'Phi' 'ENG' 'SLO' 'BAH' 'Mon']


In [ ]:
manual_country_map = {
    'RF': 'Russia',            # Possibly Russia (Russian Federation, RU), but not standard
    'EU': 'Europe',            # European Union, not a country
    'AN': 'Andorra',            # Could be Netherlands Antilles (obsolete), or Andorra (AD)
    'UR': 'Uruguay',            # Possibly Uruguay (UY)
    'Ire': 'Ireland',
    'Sve': 'Sweden',           # Possibly Sweden (SE)
    'YS': 'United States',            # Unclear
    'Mor': 'Morocco',
    'KYA': 'Kenya',
    'Mal': 'Malaysia',     # Could also be Malta, context needed
    'TA': 'Taiwan',            # Unclear
    'SUD': 'Sudan',
    'USp': 'United States',# Likely typo for US
    'MI': 'United States',            # Could be Michigan (US state), or Malawi (MW)
    'IRE': 'Ireland',
    'dil': 'United States',           # Unclear
    'OH': 'United States',            # Ohio (US state), not a country
    'Rep': 'Dominican Republic',           # Republic of..., ambiguous
    'UAE': 'United Arab Emirates',
    'Por': 'Portugal',
    'RSA': 'South Africa', # Republic of South Africa
    'CT': 'United States',            # Connecticut (US state) or Cape Town (city)
    'Rom': 'Romania',
    'Cou': 'United States',           # Could be "Country", unclear
    'NED': 'Netherlands',
    'AXA': 'Anguilla',
    'VIC': 'Australia',           # Victoria (AU state), not a country
    'UU': 'United States',            # Unclear
    'SP': 'Spain',
    'MRU': 'Mauritania',
    'FL': 'United States',            # Florida (US state), not a country
    'AUA': 'Australia',           # Aruba (AW), but not standard
    'Net': 'Netherlands',
    'Ser': 'Serbia',
    'GER': 'Germany',
    'Sin': 'Singapore',
    'CRO': 'Croatia',
    'PHI': 'Philippines',
    'BNG': 'Bangladesh',
    'QR': 'Qatar',
    'Lh': 'United States',            # Unclear
    'JA': 'Jamaica',
    'USE': 'United States',
    'USI': 'United States',
    'NIG': 'Nigeria',
    'Vic': 'Australia',           # Victoria (AU state), not a country
    'Ger': 'Germany',
    'Php': 'Philippines',
    'GAM': 'Gambia',
    'WA': 'United States',            # Western Australia (state), not a country
    'Phi': 'Philippines',
    'ENG': 'United Kingdom', # England, use UK for country-level
    'SLO': 'Slovenia',
    'BAH': 'Bahamas',
    'Mon': 'Monaco',
    # Handle 'nan' and empty as missing
    None: None,
    'nan': None,
    '': None
}


In [ ]:
df.loc[df['country_name'].isnull(), 'country_name'] = (
    df.loc[df['country_name'].isnull(), 'user_country'].map(manual_country_map)
)


In [ ]:
# Show user_country codes that failed to convert (for manual review)
failed_codes = df[df['country_name'].isnull()]['user_country'].unique()
print("Failed codes:", failed_codes)

Failed codes: [nan]


In [ ]:
df['country_name'].unique()

array(['United States', 'Australia', 'Germany', 'Canada',
       'United Kingdom', 'Romania', 'Cyprus', nan, 'New Zealand',
       'Russia', 'Ireland', 'Saudi Arabia', 'Spain', 'Samoa', 'Europe',
       'Denmark', 'Greece', 'Hungary', 'India', 'Sweden', 'Portugal',
       'Iran', 'Finland', 'Netherlands', 'Timor-Leste', 'France',
       'Albania', 'Andorra', 'South Africa', 'Ukraine', 'Nicaragua',
       'Indonesia', 'Peru', 'Colombia', 'Uruguay', 'Mexico', 'Morocco',
       'Norway', 'Japan', 'South Korea', 'Philippines', 'Kenya',
       'Malaysia', 'Guam', 'Taiwan', 'Macau', 'Sudan', 'Slovenia',
       'Iceland', 'Italy', 'American Samoa', 'Pakistan',
       'Dominican Republic', 'United Arab Emirates', 'Singapore',
       'Switzerland', 'Georgia', 'Anguilla', 'Thailand', 'Bhutan',
       'Mauritania', 'Türkiye', 'Serbia', 'Argentina', 'Czechia',
       'Croatia', 'Bangladesh', 'Qatar', 'New Caledonia',
       'North Macedonia', 'Belgium', 'Vietnam', 'Jamaica', 'Nigeria',
       'El 

In [ ]:
# Filter the DataFrame to show rows where 'user_country' is null
null_user_country_rows = df[df['country_name'].isnull()]

print("Rows where user_country is null:")
display(null_user_country_rows)

Rows where user_country is null:


,thread_id,thread_title,thread_url,forum_category,thread_views,thread_replies,total_posts,post_type,post_index,post_id,...,user_joined,user_messages,user_country,user_state,has_quotes,quote_count,all_quotes,quoted_users,timestamp_parsed,country_name
64,tremor-and-loss-of-hand-dexterity-and-strength...,Tremor and loss of hand dexterity and strength,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,501,36,37,reply,8,post-329298,...,NaN,NaN,NaN,NaN,No,0,NaN,NaN,2025-05-11 10:46:33-04:00,NaN
66,tremor-and-loss-of-hand-dexterity-and-strength...,Tremor and loss of hand dexterity and strength,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,501,36,37,reply,10,post-329307,...,NaN,NaN,NaN,NaN,No,0,NaN,NaN,2025-05-11 16:17:02-04:00,NaN
69,tremor-and-loss-of-hand-dexterity-and-strength...,Tremor and loss of hand dexterity and strength,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,501,36,37,reply,13,post-329316,...,"Apr 23, 2025",4,NaN,NaN,No,0,NaN,NaN,2025-05-12 11:01:32-04:00,NaN
70,tremor-and-loss-of-hand-dexterity-and-strength...,Tremor and loss of hand dexterity and strength,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,501,36,37,reply,14,post-329338,...,"May 9, 2025",4,NaN,NaN,No,0,NaN,NaN,2025-05-13 07:52:47-04:00,NaN
71,tremor-and-loss-of-hand-dexterity-and-strength...,Tremor and loss of hand dexterity and strength,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,501,36,37,reply,15,post-329339,...,"May 9, 2025",4,NaN,NaN,No,0,NaN,NaN,2025-05-13 07:54:04-04:00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66630,biking.28701,Biking!,https://www.fibromyalgiaforums.org/community/t...,Hobbies and interests,2K,27,28,reply,9,post-308023,...,NaN,NaN,NaN,NaN,No,0,NaN,NaN,2021-08-23 03:36:31-04:00,NaN
66635,biking.28701,Biking!,https://www.fibromyalgiaforums.org/community/t...,Hobbies and interests,2K,27,28,reply,14,post-308096,...,NaN,NaN,NaN,NaN,No,0,NaN,NaN,2021-08-24 13:54:51-04:00,NaN
66640,biking.28701,Biking!,https://www.fibromyalgiaforums.org/community/t...,Hobbies and interests,2K,27,28,reply,19,post-308141,...,NaN,NaN,NaN,NaN,Yes,2,Jemima said:\n\t\t\t\n\t\t\n\t\n\t\n\t\t\n\t\t...,Jemima | Jemima,2021-08-26 14:36:33-04:00,NaN
66644,biking.28701,Biking!,https://www.fibromyalgiaforums.org/community/t...,Hobbies and interests,2K,27,28,reply,23,post-308523,...,NaN,NaN,NaN,NaN,No,0,NaN,NaN,2021-09-14 18:24:47-04:00,NaN


In [ ]:
missing_count = df['user_country'].isnull().sum()
missing_percent = df['user_country'].isnull().mean() * 100
print(f"Missing user_country count: {missing_count} ({missing_percent:.2f}%)")


Missing user_country count: 4370 (6.51%)


In [ ]:
df['user_country'] = df['user_country'].fillna('Unknown')


In [ ]:
missing_count = df['user_country'].isnull().sum()
missing_percent = df['user_country'].isnull().mean() * 100
print(f"Missing user_country count: {missing_count} ({missing_percent:.2f}%)")

Missing user_country count: 0 (0.00%)


In [ ]:
df.head()

,thread_id,thread_title,thread_url,forum_category,thread_views,thread_replies,total_posts,post_type,post_index,post_id,...,user_joined,user_messages,user_country,user_state,has_quotes,quote_count,all_quotes,quoted_users,timestamp_parsed,country_name
0,my-advice-for-managing-fibromyalgia-especially...,My advice for managing fibromyalgia (especiall...,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,27K,29,30,original_post,0,post-302771,...,"Dec 2, 2016","4,081",US,NaN,No,0,NaN,NaN,2020-07-21 14:08:05-04:00,United States
1,my-advice-for-managing-fibromyalgia-especially...,My advice for managing fibromyalgia (especiall...,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,27K,29,30,reply,1,post-302807,...,"Jun 25, 2020",74,AU,NaN,No,0,NaN,NaN,2020-07-27 07:35:41-04:00,Australia
2,my-advice-for-managing-fibromyalgia-especially...,My advice for managing fibromyalgia (especiall...,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,27K,29,30,reply,2,post-302809,...,"Feb 26, 2020",6,US,MO,No,0,NaN,NaN,2020-07-27 08:58:09-04:00,United States
3,my-advice-for-managing-fibromyalgia-especially...,My advice for managing fibromyalgia (especiall...,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,27K,29,30,reply,3,post-302813,...,"Dec 2, 2016","4,081",US,NaN,No,0,NaN,NaN,2020-07-27 10:01:34-04:00,United States
4,my-advice-for-managing-fibromyalgia-especially...,My advice for managing fibromyalgia (especiall...,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,27K,29,30,reply,4,post-302814,...,"Dec 2, 2016","4,081",US,NaN,No,0,NaN,NaN,2020-07-27 10:09:51-04:00,United States


In [ ]:
# Check unique values in the 'has_quotes' column
print("Unique values in 'has_quotes' column:", df['has_quotes'].unique())

# Filter the DataFrame to show rows where 'has_quotes' is Yes
posts_with_quotes = df[df['has_quotes'] == 'Yes']

print("\nRows where 'has_quotes' is Yes:")
display(posts_with_quotes)

print(f"\nNumber of rows where 'has_quotes' is Yes: {posts_with_quotes.shape[0]}")

Unique values in 'has_quotes' column: ['No' 'Yes']

Rows where 'has_quotes' is Yes:


,thread_id,thread_title,thread_url,forum_category,thread_views,thread_replies,total_posts,post_type,post_index,post_id,...,user_joined,user_messages,user_country,user_state,has_quotes,quote_count,all_quotes,quoted_users,timestamp_parsed,country_name
11,my-advice-for-managing-fibromyalgia-especially...,My advice for managing fibromyalgia (especiall...,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,27K,29,30,reply,11,post-303569,...,"Dec 2, 2016","4,081",US,NaN,Yes,2,Drummer76 said:\n\t\t\t\n\t\t\n\t\n\t\n\t\t\n\...,Drummer76 | Drummer76,2020-09-14 10:11:04-04:00,United States
13,my-advice-for-managing-fibromyalgia-especially...,My advice for managing fibromyalgia (especiall...,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,27K,29,30,reply,13,post-303708,...,"Dec 2, 2016","4,081",US,NaN,Yes,2,DonaldKBradley said:\n\t\t\t\n\t\t\n\t\n\t\n\t...,DonaldKBradley | DonaldKBradley,2020-09-23 13:03:41-04:00,United States
22,my-advice-for-managing-fibromyalgia-especially...,My advice for managing fibromyalgia (especiall...,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,27K,29,30,reply,22,post-304891,...,"Dec 2, 2016","4,081",US,NaN,Yes,2,gringasalsera said:\n\t\t\t\n\t\t\n\t\n\t\n\t\...,gringasalsera | gringasalsera,2021-01-03 09:02:47-05:00,United States
24,my-advice-for-managing-fibromyalgia-especially...,My advice for managing fibromyalgia (especiall...,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,27K,29,30,reply,24,post-305743,...,"Aug 2, 2020",65,US,NJ,Yes,2,sunkacola said:\n\t\t\t\n\t\t\n\t\n\t\n\t\t\n\...,sunkacola | sunkacola,2021-03-29 11:39:37-04:00,United States
25,my-advice-for-managing-fibromyalgia-especially...,My advice for managing fibromyalgia (especiall...,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,27K,29,30,reply,25,post-305767,...,"Dec 2, 2016","4,081",US,NaN,Yes,2,srdodla123 said:\n\t\t\t\n\t\t\n\t\n\t\n\t\t\n...,srdodla123 | srdodla123,2021-03-30 11:12:13-04:00,United States
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67041,companion-animals.21438,Companion Animals,https://www.fibromyalgiaforums.org/community/t...,Hobbies and interests,8K,17,18,reply,7,post-262576,...,"Mar 14, 2013",82,US,MI,Yes,2,Moxie said:\n\t\t\t\n\t\t\n\t\n\t\n\t\t\n\t\t\...,Moxie | Moxie,2013-03-30 00:49:13-04:00,United States
67057,profiting-from-something-that-you-love-doing.2...,Profiting from something that you love doing,https://www.fibromyalgiaforums.org/community/t...,Hobbies and interests,4K,14,15,reply,5,post-264638,...,"Jun 20, 2013",49,ro,romania,Yes,2,Libragirl67 said:\n\t\t\t\n\t\t\n\t\n\t\n\t\t\...,Libragirl67 | Libragirl67,2013-07-06 11:26:04-04:00,Romania
67086,the-joy-of-cooking.21580,the joy of cooking,https://www.fibromyalgiaforums.org/community/t...,Hobbies and interests,6K,17,18,reply,11,post-263307,...,"Apr 19, 2013",459,US,PA,Yes,2,BrianWolfe said:\n\t\t\t\n\t\t\n\t\n\t\n\t\t\n...,BrianWolfe | BrianWolfe,2013-04-26 13:07:43-04:00,United States
67107,the-joy-of-bird-watching.21446,The Joy Of Bird Watching,https://www.fibromyalgiaforums.org/community/t...,Hobbies and interests,3K,13,14,reply,9,post-263240,...,"Mar 14, 2013",82,US,MI,Yes,2,schizophreniatype said:\n\t\t\t\n\t\t\n\t\n\t\...,schizophreniatype | schizophreniatype,2013-04-24 17:19:27-04:00,United States



Number of rows where 'has_quotes' is Yes: 10562


##Function to clean numeric strings

In [ ]:
# --- Function to clean numeric strings ---
def clean_numeric_value(value):
    """Cleans strings like '1,500' or '3K' into numbers."""
    if pd.isna(value):
        return np.nan
    try:
        s_value = str(value).strip().lower()
        if 'k' in s_value:
            return float(s_value.replace('k', '')) * 1000
        return float(re.sub(r'[^\d.]', '', s_value))
    except (ValueError, TypeError):
        return np.nan

print("⚙️ Correcting data types...")

# --- Convert datetime columns ---
print("   - Converting 'timestamp' and 'user_joined' to datetime objects...")
# Handle potential mixed time zones by setting utc=True
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce', utc=True)
df['user_joined'] = pd.to_datetime(df['user_joined'], errors='coerce')


# --- Clean and convert numeric columns ---
print("   - Cleaning and converting numeric columns...")
numeric_cols = ['thread_views', 'thread_replies', 'user_messages', 'quote_count', 'content_length']
for col in numeric_cols:
    if col in df.columns:
        print(f"     - Processing '{col}'...")
        # Use .loc to avoid SettingWithCopyWarning
        df.loc[:, col] = df[col].apply(clean_numeric_value)

# Convert to nullable integer type
df[numeric_cols] = df[numeric_cols].astype('Int64')

# --- Convert boolean column ---
print("   - Converting 'has_quotes' to boolean...")
if 'has_quotes' in df.columns:
    df['has_quotes'] = df['has_quotes'].map({'Yes': True, 'No': False})


print("\n✅ Data types corrected. Let's verify:")
print(df.dtypes)

⚙️ Correcting data types...
   - Converting 'timestamp' and 'user_joined' to datetime objects...
   - Cleaning and converting numeric columns...
     - Processing 'thread_views'...
     - Processing 'thread_replies'...
     - Processing 'user_messages'...
     - Processing 'quote_count'...
     - Processing 'content_length'...
   - Converting 'has_quotes' to boolean...

✅ Data types corrected. Let's verify:
thread_id                        object
thread_title                     object
thread_url                       object
forum_category                   object
thread_views                      Int64
thread_replies                    Int64
total_posts                       int64
post_type                        object
post_index                        int64
post_id                          object
username                         object
user_title                       object
timestamp           datetime64[ns, UTC]
content                          object
content_length               

In [ ]:
df.head()

,thread_id,thread_title,thread_url,forum_category,thread_views,thread_replies,total_posts,post_type,post_index,post_id,...,user_joined,user_messages,user_country,user_state,has_quotes,quote_count,all_quotes,quoted_users,timestamp_parsed,country_name
0,my-advice-for-managing-fibromyalgia-especially...,My advice for managing fibromyalgia (especiall...,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,27000,29,30,original_post,0,post-302771,...,2016-12-02,4081,US,NaN,False,0,NaN,NaN,2020-07-21 14:08:05-04:00,United States
1,my-advice-for-managing-fibromyalgia-especially...,My advice for managing fibromyalgia (especiall...,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,27000,29,30,reply,1,post-302807,...,2020-06-25,74,AU,NaN,False,0,NaN,NaN,2020-07-27 07:35:41-04:00,Australia
2,my-advice-for-managing-fibromyalgia-especially...,My advice for managing fibromyalgia (especiall...,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,27000,29,30,reply,2,post-302809,...,2020-02-26,6,US,MO,False,0,NaN,NaN,2020-07-27 08:58:09-04:00,United States
3,my-advice-for-managing-fibromyalgia-especially...,My advice for managing fibromyalgia (especiall...,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,27000,29,30,reply,3,post-302813,...,2016-12-02,4081,US,NaN,False,0,NaN,NaN,2020-07-27 10:01:34-04:00,United States
4,my-advice-for-managing-fibromyalgia-especially...,My advice for managing fibromyalgia (especiall...,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,27000,29,30,reply,4,post-302814,...,2016-12-02,4081,US,NaN,False,0,NaN,NaN,2020-07-27 10:09:51-04:00,United States


In [ ]:
# What proportion of posts include quotes?
quote_rate = df['has_quotes'].mean() * 100
print(f'Percentage of posts with quotes: {quote_rate:.2f}%')


Percentage of posts with quotes: 15.73%


## Check for rows with missing Content

In [ ]:
# Check for rows where 'content' is NaN or empty/whitespace
missing_content_rows = df[df['content'].isnull() | (df['content'].str.strip() == '')]

# To see how many such rows there are:
print(f"Number of rows with missing or empty content: {missing_content_rows.shape[0]}")

# To view the actual rows (optional, e.g., first 5)
# print(missing_content_rows.head())


Number of rows with missing or empty content: 44


In [ ]:
# Drop rows where content is missing (critical for text analysis)
df = df.dropna(subset=['content'])

## Check for rows with missing Joined Date

In [ ]:
# Find rows where 'user_joined' is missing (NaN)
missing_user_joined = df[df['user_joined'].isnull()]

# See how many rows are missing
print(f"Rows with missing user_joined: {missing_user_joined.shape[0]}")

# Display the entire DataFrame with missing 'user_joined'
display(missing_user_joined)

Rows with missing user_joined: 1406


,thread_id,thread_title,thread_url,forum_category,thread_views,thread_replies,total_posts,post_type,post_index,post_id,...,user_joined,user_messages,user_country,user_state,has_quotes,quote_count,all_quotes,quoted_users,timestamp_parsed,country_name
64,tremor-and-loss-of-hand-dexterity-and-strength...,Tremor and loss of hand dexterity and strength,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,501,36,37,reply,8,post-329298,...,NaT,<NA>,Unknown,NaN,False,0,NaN,NaN,2025-05-11 10:46:33-04:00,NaN
66,tremor-and-loss-of-hand-dexterity-and-strength...,Tremor and loss of hand dexterity and strength,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,501,36,37,reply,10,post-329307,...,NaT,<NA>,Unknown,NaN,False,0,NaN,NaN,2025-05-11 16:17:02-04:00,NaN
3803,late-period-arrival-and-pms.29891,Late period arrival (and PMS),https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,359,1,2,original_post,0,post-320339,...,NaT,<NA>,Unknown,NaN,False,0,NaN,NaN,2023-06-07 18:31:09-04:00,NaN
3805,symptoms-come-and-go.29883,Symptoms come and go?,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,516,5,6,original_post,0,post-320265,...,NaT,<NA>,Unknown,NaN,False,0,NaN,NaN,2023-06-03 08:45:29-04:00,NaN
3808,symptoms-come-and-go.29883,Symptoms come and go?,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,516,5,6,reply,3,post-320270,...,NaT,<NA>,Unknown,NaN,False,0,NaN,NaN,2023-06-03 11:56:39-04:00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66630,biking.28701,Biking!,https://www.fibromyalgiaforums.org/community/t...,Hobbies and interests,2000,27,28,reply,9,post-308023,...,NaT,<NA>,Unknown,NaN,False,0,NaN,NaN,2021-08-23 03:36:31-04:00,NaN
66635,biking.28701,Biking!,https://www.fibromyalgiaforums.org/community/t...,Hobbies and interests,2000,27,28,reply,14,post-308096,...,NaT,<NA>,Unknown,NaN,False,0,NaN,NaN,2021-08-24 13:54:51-04:00,NaN
66640,biking.28701,Biking!,https://www.fibromyalgiaforums.org/community/t...,Hobbies and interests,2000,27,28,reply,19,post-308141,...,NaT,<NA>,Unknown,NaN,True,2,Jemima said:\n\t\t\t\n\t\t\n\t\n\t\n\t\t\n\t\t...,Jemima | Jemima,2021-08-26 14:36:33-04:00,NaN
66644,biking.28701,Biking!,https://www.fibromyalgiaforums.org/community/t...,Hobbies and interests,2000,27,28,reply,23,post-308523,...,NaT,<NA>,Unknown,NaN,False,0,NaN,NaN,2021-09-14 18:24:47-04:00,NaN


In [ ]:
# Filter the missing_user_joined DataFrame to find rows where user_title is 'Guest'
guest_missing_user_joined = missing_user_joined[missing_user_joined['user_title'] == 'Guest']

print("Rows with missing user_joined and user_title as 'Guest':")
display(guest_missing_user_joined)

print(f"\nNumber of rows with missing user_joined and user_title as 'Guest': {guest_missing_user_joined.shape[0]}")

Rows with missing user_joined and user_title as 'Guest':


,thread_id,thread_title,thread_url,forum_category,thread_views,thread_replies,total_posts,post_type,post_index,post_id,...,user_joined,user_messages,user_country,user_state,has_quotes,quote_count,all_quotes,quoted_users,timestamp_parsed,country_name
64,tremor-and-loss-of-hand-dexterity-and-strength...,Tremor and loss of hand dexterity and strength,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,501,36,37,reply,8,post-329298,...,NaT,<NA>,Unknown,NaN,False,0,NaN,NaN,2025-05-11 10:46:33-04:00,NaN
66,tremor-and-loss-of-hand-dexterity-and-strength...,Tremor and loss of hand dexterity and strength,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,501,36,37,reply,10,post-329307,...,NaT,<NA>,Unknown,NaN,False,0,NaN,NaN,2025-05-11 16:17:02-04:00,NaN
3803,late-period-arrival-and-pms.29891,Late period arrival (and PMS),https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,359,1,2,original_post,0,post-320339,...,NaT,<NA>,Unknown,NaN,False,0,NaN,NaN,2023-06-07 18:31:09-04:00,NaN
3805,symptoms-come-and-go.29883,Symptoms come and go?,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,516,5,6,original_post,0,post-320265,...,NaT,<NA>,Unknown,NaN,False,0,NaN,NaN,2023-06-03 08:45:29-04:00,NaN
3808,symptoms-come-and-go.29883,Symptoms come and go?,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,516,5,6,reply,3,post-320270,...,NaT,<NA>,Unknown,NaN,False,0,NaN,NaN,2023-06-03 11:56:39-04:00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66630,biking.28701,Biking!,https://www.fibromyalgiaforums.org/community/t...,Hobbies and interests,2000,27,28,reply,9,post-308023,...,NaT,<NA>,Unknown,NaN,False,0,NaN,NaN,2021-08-23 03:36:31-04:00,NaN
66635,biking.28701,Biking!,https://www.fibromyalgiaforums.org/community/t...,Hobbies and interests,2000,27,28,reply,14,post-308096,...,NaT,<NA>,Unknown,NaN,False,0,NaN,NaN,2021-08-24 13:54:51-04:00,NaN
66640,biking.28701,Biking!,https://www.fibromyalgiaforums.org/community/t...,Hobbies and interests,2000,27,28,reply,19,post-308141,...,NaT,<NA>,Unknown,NaN,True,2,Jemima said:\n\t\t\t\n\t\t\n\t\n\t\n\t\t\n\t\t...,Jemima | Jemima,2021-08-26 14:36:33-04:00,NaN
66644,biking.28701,Biking!,https://www.fibromyalgiaforums.org/community/t...,Hobbies and interests,2000,27,28,reply,23,post-308523,...,NaT,<NA>,Unknown,NaN,False,0,NaN,NaN,2021-09-14 18:24:47-04:00,NaN



Number of rows with missing user_joined and user_title as 'Guest': 1406


In [ ]:
# Filter the missing_user_joined DataFrame to find rows where user_title is NOT 'Guest'
not_guest_missing_user_joined = missing_user_joined[missing_user_joined['user_title'] != 'Guest']

print("Rows with missing user_joined where user_title is NOT 'Guest':")
display(not_guest_missing_user_joined)

print(f"\nNumber of rows with missing user_joined and user_title is NOT 'Guest': {not_guest_missing_user_joined.shape[0]}")

Rows with missing user_joined where user_title is NOT 'Guest':


,thread_id,thread_title,thread_url,forum_category,thread_views,thread_replies,total_posts,post_type,post_index,post_id,...,user_joined,user_messages,user_country,user_state,has_quotes,quote_count,all_quotes,quoted_users,timestamp_parsed,country_name



Number of rows with missing user_joined and user_title is NOT 'Guest': 0


In [ ]:
# Filter the missing_user_joined DataFrame to find rows where username contains 'Deleted member'
deleted_member_missing_user_joined = missing_user_joined[missing_user_joined['username'].str.contains('Deleted member', na=False)]

print("Rows with missing user_joined and username containing 'Deleted member':")
display(deleted_member_missing_user_joined)

print(f"\nNumber of rows with missing user_joined and username containing 'Deleted member': {deleted_member_missing_user_joined.shape[0]}")

Rows with missing user_joined and username containing 'Deleted member':


,thread_id,thread_title,thread_url,forum_category,thread_views,thread_replies,total_posts,post_type,post_index,post_id,...,user_joined,user_messages,user_country,user_state,has_quotes,quote_count,all_quotes,quoted_users,timestamp_parsed,country_name
64,tremor-and-loss-of-hand-dexterity-and-strength...,Tremor and loss of hand dexterity and strength,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,501,36,37,reply,8,post-329298,...,NaT,<NA>,Unknown,NaN,False,0,NaN,NaN,2025-05-11 10:46:33-04:00,NaN
66,tremor-and-loss-of-hand-dexterity-and-strength...,Tremor and loss of hand dexterity and strength,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,501,36,37,reply,10,post-329307,...,NaT,<NA>,Unknown,NaN,False,0,NaN,NaN,2025-05-11 16:17:02-04:00,NaN
3803,late-period-arrival-and-pms.29891,Late period arrival (and PMS),https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,359,1,2,original_post,0,post-320339,...,NaT,<NA>,Unknown,NaN,False,0,NaN,NaN,2023-06-07 18:31:09-04:00,NaN
3805,symptoms-come-and-go.29883,Symptoms come and go?,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,516,5,6,original_post,0,post-320265,...,NaT,<NA>,Unknown,NaN,False,0,NaN,NaN,2023-06-03 08:45:29-04:00,NaN
3808,symptoms-come-and-go.29883,Symptoms come and go?,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,516,5,6,reply,3,post-320270,...,NaT,<NA>,Unknown,NaN,False,0,NaN,NaN,2023-06-03 11:56:39-04:00,NaN
3810,symptoms-come-and-go.29883,Symptoms come and go?,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,516,5,6,reply,5,post-320340,...,NaT,<NA>,Unknown,NaN,False,0,NaN,NaN,2023-06-07 20:18:03-04:00,NaN
3811,stem-cells-and-heartbreak.29870,Stem cells and heartbreak,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,586,3,4,original_post,0,post-320188,...,NaT,<NA>,Unknown,NaN,False,0,NaN,NaN,2023-05-30 21:19:02-04:00,NaN
3812,stem-cells-and-heartbreak.29870,Stem cells and heartbreak,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,586,3,4,reply,1,post-320189,...,NaT,<NA>,Unknown,NaN,False,0,NaN,NaN,2023-05-30 21:24:36-04:00,NaN
3814,stem-cells-and-heartbreak.29870,Stem cells and heartbreak,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,586,3,4,reply,3,post-320212,...,NaT,<NA>,Unknown,NaN,True,2,sunkacola said:\n\t\t\t\n\t\t\n\t\n\t\n\t\t\n\...,sunkacola | sunkacola,2023-05-31 22:45:09-04:00,NaN
27672,feeling-not-listened-to.30729,Feeling not listened to,https://www.fibromyalgiaforums.org/community/t...,Do I have fibromyalgia?,327,6,7,original_post,0,post-328279,...,NaT,<NA>,Unknown,NaN,False,0,NaN,NaN,2025-02-23 12:58:00-05:00,NaN



Number of rows with missing user_joined and username containing 'Deleted member': 15


In [ ]:
# Filter the missing_user_joined DataFrame to find rows where username does NOT contain 'Deleted member'
not_deleted_member_missing_user_joined = missing_user_joined[~missing_user_joined['username'].str.contains('Deleted member', na=False)]

print("Rows with missing user_joined and username does NOT contain 'Deleted member':")
display(not_deleted_member_missing_user_joined)

print(f"\nNumber of rows with missing user_joined and username does NOT contain 'Deleted member': {not_deleted_member_missing_user_joined.shape[0]}")

Rows with missing user_joined and username does NOT contain 'Deleted member':


,thread_id,thread_title,thread_url,forum_category,thread_views,thread_replies,total_posts,post_type,post_index,post_id,...,user_joined,user_messages,user_country,user_state,has_quotes,quote_count,all_quotes,quoted_users,timestamp_parsed,country_name
6680,remember-fibromyalgia-is-a-syndrome.28693,Remember Fibromyalgia is a Syndrome,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,3000,13,14,reply,4,post-307616,...,NaT,<NA>,Unknown,NaN,False,0,NaN,NaN,2021-08-01 12:29:46-04:00,NaN
6742,where-is-the-weirdest-place-you-hurt-what-are-...,Where is the weirdest place you hurt? What are...,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,8000,56,57,reply,21,post-308739,...,NaT,<NA>,Unknown,NaN,True,2,CScalise said:\n\t\t\t\n\t\t\n\t\n\t\n\t\t\n\t...,CScalise | CScalise,2021-09-27 07:21:16-04:00,NaN
7583,cannot-sleep-again-lets-join-the-wide-awake-cl...,Cannot sleep again lets join the wide awake club.,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,1000,8,9,reply,2,post-308604,...,NaT,<NA>,Unknown,NaN,False,0,NaN,NaN,2021-09-19 17:12:12-04:00,NaN
7606,wobbly-legs.28754,Wobbly Legs,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,2000,17,18,reply,16,post-308524,...,NaT,<NA>,Unknown,NaN,True,2,JayCS said:\n\t\t\t\n\t\t\n\t\n\t\n\t\t\n\t\t\...,JayCS | JayCS,2021-09-14 18:29:32-04:00,NaN
7669,hi-im-new-here-and-the-last-month-has-been-a-r...,"Hi, I'm new here and the last month has been a...",https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,1000,6,7,reply,2,post-308700,...,NaT,<NA>,Unknown,NaN,False,0,NaN,NaN,2021-09-25 17:10:39-04:00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66630,biking.28701,Biking!,https://www.fibromyalgiaforums.org/community/t...,Hobbies and interests,2000,27,28,reply,9,post-308023,...,NaT,<NA>,Unknown,NaN,False,0,NaN,NaN,2021-08-23 03:36:31-04:00,NaN
66635,biking.28701,Biking!,https://www.fibromyalgiaforums.org/community/t...,Hobbies and interests,2000,27,28,reply,14,post-308096,...,NaT,<NA>,Unknown,NaN,False,0,NaN,NaN,2021-08-24 13:54:51-04:00,NaN
66640,biking.28701,Biking!,https://www.fibromyalgiaforums.org/community/t...,Hobbies and interests,2000,27,28,reply,19,post-308141,...,NaT,<NA>,Unknown,NaN,True,2,Jemima said:\n\t\t\t\n\t\t\n\t\n\t\n\t\t\n\t\t...,Jemima | Jemima,2021-08-26 14:36:33-04:00,NaN
66644,biking.28701,Biking!,https://www.fibromyalgiaforums.org/community/t...,Hobbies and interests,2000,27,28,reply,23,post-308523,...,NaT,<NA>,Unknown,NaN,False,0,NaN,NaN,2021-09-14 18:24:47-04:00,NaN



Number of rows with missing user_joined and username does NOT contain 'Deleted member': 1391


In [ ]:
# Filter the missing_user_joined DataFrame to find rows where username does NOT contain 'Deleted member'
not_deleted_member_missing_user_joined = missing_user_joined[missing_user_joined['username'].str.contains('User Requested Deletion', na=False)]

print("Rows with missing user_joined and username does NOT contain 'Deleted member':")
display(not_deleted_member_missing_user_joined)

print(f"\nNumber of rows with missing user_joined and username does NOT contain 'Deleted member': {not_deleted_member_missing_user_joined.shape[0]}")

Rows with missing user_joined and username does NOT contain 'Deleted member':


,thread_id,thread_title,thread_url,forum_category,thread_views,thread_replies,total_posts,post_type,post_index,post_id,...,user_joined,user_messages,user_country,user_state,has_quotes,quote_count,all_quotes,quoted_users,timestamp_parsed,country_name
6680,remember-fibromyalgia-is-a-syndrome.28693,Remember Fibromyalgia is a Syndrome,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,3000,13,14,reply,4,post-307616,...,NaT,<NA>,Unknown,NaN,False,0,NaN,NaN,2021-08-01 12:29:46-04:00,NaN
6742,where-is-the-weirdest-place-you-hurt-what-are-...,Where is the weirdest place you hurt? What are...,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,8000,56,57,reply,21,post-308739,...,NaT,<NA>,Unknown,NaN,True,2,CScalise said:\n\t\t\t\n\t\t\n\t\n\t\n\t\t\n\t...,CScalise | CScalise,2021-09-27 07:21:16-04:00,NaN
7583,cannot-sleep-again-lets-join-the-wide-awake-cl...,Cannot sleep again lets join the wide awake club.,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,1000,8,9,reply,2,post-308604,...,NaT,<NA>,Unknown,NaN,False,0,NaN,NaN,2021-09-19 17:12:12-04:00,NaN
7606,wobbly-legs.28754,Wobbly Legs,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,2000,17,18,reply,16,post-308524,...,NaT,<NA>,Unknown,NaN,True,2,JayCS said:\n\t\t\t\n\t\t\n\t\n\t\n\t\t\n\t\t\...,JayCS | JayCS,2021-09-14 18:29:32-04:00,NaN
7669,hi-im-new-here-and-the-last-month-has-been-a-r...,"Hi, I'm new here and the last month has been a...",https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,1000,6,7,reply,2,post-308700,...,NaT,<NA>,Unknown,NaN,False,0,NaN,NaN,2021-09-25 17:10:39-04:00,NaN
7677,challenging-time.28760,Challenging time,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,2000,16,17,reply,3,post-308271,...,NaT,<NA>,Unknown,NaN,True,2,Jemima said:\n\t\t\t\n\t\t\n\t\n\t\n\t\t\n\t\t...,Jemima | Jemima,2021-09-02 17:20:56-04:00,NaN
7686,challenging-time.28760,Challenging time,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,2000,16,17,reply,12,post-308378,...,NaT,<NA>,Unknown,NaN,False,0,NaN,NaN,2021-09-08 02:39:27-04:00,NaN
7696,hi-everyone-coping-with-fybro-is-a-new-channel...,"hi Everyone ""Coping with Fybro"" is a new Chann...",https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,3000,35,36,reply,5,post-308441,...,NaT,<NA>,Unknown,NaN,False,0,NaN,NaN,2021-09-12 11:42:43-04:00,NaN
7745,skin-hot-to-the-touch.28759,skin hot to the touch,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,2000,8,9,reply,4,post-308298,...,NaT,<NA>,Unknown,NaN,False,0,NaN,NaN,2021-09-05 14:51:48-04:00,NaN
7751,bedding.28753,Bedding,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,996,7,8,reply,1,post-308231,...,NaT,<NA>,Unknown,NaN,False,0,NaN,NaN,2021-08-31 16:21:59-04:00,NaN



Number of rows with missing user_joined and username does NOT contain 'Deleted member': 52


In [ ]:
# Filter the missing_user_joined DataFrame to exclude rows where username contains 'Deleted member' or 'User Requested Deletion'
filtered_missing_user_joined = missing_user_joined[
    (~missing_user_joined['username'].str.contains('Deleted member', na=False)) &
    (~missing_user_joined['username'].str.contains('User Requested Deletion', na=False))
]

print("Rows with missing user_joined and username does NOT contain 'Deleted member' or 'User Requested Deletion':")
display(filtered_missing_user_joined)

print(f"\nNumber of rows with missing user_joined and username does NOT contain 'Deleted member' or 'User Requested Deletion': {filtered_missing_user_joined.shape[0]}")

Rows with missing user_joined and username does NOT contain 'Deleted member' or 'User Requested Deletion':


,thread_id,thread_title,thread_url,forum_category,thread_views,thread_replies,total_posts,post_type,post_index,post_id,...,user_joined,user_messages,user_country,user_state,has_quotes,quote_count,all_quotes,quoted_users,timestamp_parsed,country_name
26369,fibromyalgia.20955,Fibromyalgia?,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,15000,11,12,original_post,0,post-258141,...,NaT,<NA>,Unknown,NaN,False,0,NaN,NaN,2012-12-06 01:50:40-05:00,NaN
26370,fibromyalgia.20955,Fibromyalgia?,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,15000,11,12,reply,1,post-258153,...,NaT,<NA>,Unknown,NaN,False,0,NaN,NaN,2012-12-06 10:32:26-05:00,NaN
26371,fibromyalgia.20955,Fibromyalgia?,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,15000,11,12,reply,2,post-258315,...,NaT,<NA>,Unknown,NaN,False,0,NaN,NaN,2012-12-09 18:59:47-05:00,NaN
26372,fibromyalgia.20955,Fibromyalgia?,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,15000,11,12,reply,3,post-258316,...,NaT,<NA>,Unknown,NaN,False,0,NaN,NaN,2012-12-09 19:19:20-05:00,NaN
26373,fibromyalgia.20955,Fibromyalgia?,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,15000,11,12,reply,4,post-258414,...,NaT,<NA>,Unknown,NaN,False,0,NaN,NaN,2012-12-10 17:38:24-05:00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50924,dear-friends.481,Dear Friends,https://www.fibromyalgiaforums.org/community/t...,Chronic fatigue,4000,2,3,reply,2,post-3467,...,NaT,<NA>,Unknown,NaN,False,0,NaN,NaN,2005-08-28 15:18:10-04:00,NaN
50925,so-glad-i-found-i-found-this-group.482,So glad I found I found this group,https://www.fibromyalgiaforums.org/community/t...,Chronic fatigue,5000,3,4,original_post,0,post-3447,...,NaT,<NA>,Unknown,NaN,False,0,NaN,NaN,2005-08-26 12:49:45-04:00,NaN
50926,so-glad-i-found-i-found-this-group.482,So glad I found I found this group,https://www.fibromyalgiaforums.org/community/t...,Chronic fatigue,5000,3,4,reply,1,post-3449,...,NaT,<NA>,Unknown,NaN,False,0,NaN,NaN,2005-08-26 18:42:35-04:00,NaN
50927,so-glad-i-found-i-found-this-group.482,So glad I found I found this group,https://www.fibromyalgiaforums.org/community/t...,Chronic fatigue,5000,3,4,reply,2,post-3450,...,NaT,<NA>,Unknown,NaN,True,2,Is it the flu - or Lyme disease?\nLarissa Lytw...,Unknown | Unknown,2005-08-26 21:56:54-04:00,NaN



Number of rows with missing user_joined and username does NOT contain 'Deleted member' or 'User Requested Deletion': 1339


In [ ]:
# Filter the DataFrame to show rows where user_title is 'Guest'
guest_users_df = df[df['user_title'] == 'Guest']

print("Rows where user_title is 'Guest':")
display(guest_users_df.head())

print(f"\nNumber of rows where user_title is 'Guest': {guest_users_df.shape[0]}")

Rows where user_title is 'Guest':


,thread_id,thread_title,thread_url,forum_category,thread_views,thread_replies,total_posts,post_type,post_index,post_id,...,user_joined,user_messages,user_country,user_state,has_quotes,quote_count,all_quotes,quoted_users,timestamp_parsed,country_name
64,tremor-and-loss-of-hand-dexterity-and-strength...,Tremor and loss of hand dexterity and strength,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,501,36,37,reply,8,post-329298,...,NaT,<NA>,Unknown,NaN,False,0,NaN,NaN,2025-05-11 10:46:33-04:00,NaN
66,tremor-and-loss-of-hand-dexterity-and-strength...,Tremor and loss of hand dexterity and strength,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,501,36,37,reply,10,post-329307,...,NaT,<NA>,Unknown,NaN,False,0,NaN,NaN,2025-05-11 16:17:02-04:00,NaN
3803,late-period-arrival-and-pms.29891,Late period arrival (and PMS),https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,359,1,2,original_post,0,post-320339,...,NaT,<NA>,Unknown,NaN,False,0,NaN,NaN,2023-06-07 18:31:09-04:00,NaN
3805,symptoms-come-and-go.29883,Symptoms come and go?,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,516,5,6,original_post,0,post-320265,...,NaT,<NA>,Unknown,NaN,False,0,NaN,NaN,2023-06-03 08:45:29-04:00,NaN
3808,symptoms-come-and-go.29883,Symptoms come and go?,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,516,5,6,reply,3,post-320270,...,NaT,<NA>,Unknown,NaN,False,0,NaN,NaN,2023-06-03 11:56:39-04:00,NaN



Number of rows where user_title is 'Guest': 1406


"Posts from guest users—defined as users without a join date and with 'Guest' as their user title—were retained in the dataset to preserve the completeness and diversity of forum discussions. These posts were flagged and excluded from analyses requiring user-level demographic or longitudinal data, but included in content and emotional tone analyses to avoid biasing results against anonymous contributions."

In [ ]:
# Create a flag for guest users
df['is_guest'] = df['user_title'] == 'Guest'

In [ ]:
# The columns user_state, all_quotes and quoted_users are mostly NaN, we replace NaNs with empty strings
df['all_quotes'] = df['all_quotes'].fillna('')
df['quoted_users'] = df['quoted_users'].fillna('')
df['user_state'] = df['user_state'].fillna('')

In [ ]:
# Format date and time columns
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
df['user_joined'] = pd.to_datetime(df['user_joined'], errors='coerce')

# Calculate user tenure (days)
df['user_tenure_days'] = (pd.Timestamp.now() - df['user_joined']).dt.days

In [ ]:
df.head()

,thread_id,thread_title,thread_url,forum_category,thread_views,thread_replies,total_posts,post_type,post_index,post_id,...,user_country,user_state,has_quotes,quote_count,all_quotes,quoted_users,timestamp_parsed,country_name,is_guest,user_tenure_days
0,my-advice-for-managing-fibromyalgia-especially...,My advice for managing fibromyalgia (especiall...,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,27000,29,30,original_post,0,post-302771,...,US,,False,0,,,2020-07-21 14:08:05-04:00,United States,False,3136.0
1,my-advice-for-managing-fibromyalgia-especially...,My advice for managing fibromyalgia (especiall...,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,27000,29,30,reply,1,post-302807,...,AU,,False,0,,,2020-07-27 07:35:41-04:00,Australia,False,1835.0
2,my-advice-for-managing-fibromyalgia-especially...,My advice for managing fibromyalgia (especiall...,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,27000,29,30,reply,2,post-302809,...,US,MO,False,0,,,2020-07-27 08:58:09-04:00,United States,False,1955.0
3,my-advice-for-managing-fibromyalgia-especially...,My advice for managing fibromyalgia (especiall...,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,27000,29,30,reply,3,post-302813,...,US,,False,0,,,2020-07-27 10:01:34-04:00,United States,False,3136.0
4,my-advice-for-managing-fibromyalgia-especially...,My advice for managing fibromyalgia (especiall...,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,27000,29,30,reply,4,post-302814,...,US,,False,0,,,2020-07-27 10:09:51-04:00,United States,False,3136.0


In [ ]:
# Recheck to confirm HTML tags have been removed
html_tag_post_clean = df['content'].apply(lambda x: bool(re.search(r'<[^>]+>', x))).sum()

html_tag_post_clean

np.int64(331)

In [ ]:
post = df[df['post_id'] == 'post-329008']
post

,thread_id,thread_title,thread_url,forum_category,thread_views,thread_replies,total_posts,post_type,post_index,post_id,...,user_country,user_state,has_quotes,quote_count,all_quotes,quoted_users,timestamp_parsed,country_name,is_guest,user_tenure_days
160,need-a-little-laugh-and-some-fun.30833,Need a little laugh and some fun,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,583,25,26,reply,1,post-329008,...,AU,NS,False,0,,,2025-04-26 09:42:19-04:00,Australia,False,1042.0


In [ ]:
from bs4 import BeautifulSoup, MarkupResemblesLocatorWarning
import warnings

warnings.filterwarnings("ignore", category=MarkupResemblesLocatorWarning)

# Function to clean HTML tags from content
def remove_html_tags(text):
    return BeautifulSoup(text, "html.parser").get_text()

# Apply the cleaning function to all content entries
df['content'] = df['content'].apply(remove_html_tags)

# Recheck to confirm HTML tags have been removed
html_tag_post_clean = df['content'].apply(lambda x: bool(re.search(r'<[^>]+>', x))).sum()

html_tag_post_clean

np.int64(9)

In [ ]:
post = df[df['post_id'] == 'post-329381']
post

,thread_id,thread_title,thread_url,forum_category,thread_views,thread_replies,total_posts,post_type,post_index,post_id,...,user_country,user_state,has_quotes,quote_count,all_quotes,quoted_users,timestamp_parsed,country_name,is_guest,user_tenure_days
170,need-a-little-laugh-and-some-fun.30833,Need a little laugh and some fun,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,583,25,26,reply,11,post-329381,...,AU,WA,False,0,,,2025-05-17 18:56:08-04:00,Australia,False,996.0


In [ ]:
df['content'].isnull().sum()

np.int64(0)

In [ ]:
# import re

# def clean_escape_sequences(text):
#     if pd.isnull(text):
#         return text
#     # Replace \n and \t with a space
#     return re.sub(r'[\n\t]+', ' ', text)

# df['content'] = df['content'].apply(clean_escape_sequences)


In [ ]:
# Recheck to confirm HTML tags have been removed
html_tag_post_clean = df['content'].apply(lambda x: bool(re.search(r'<[^>]+>', x))).sum()

html_tag_post_clean

np.int64(9)

In [ ]:
def remove_angle_brackets(text):
    if pd.isnull(text):
        return text
    return text.replace('<', '').replace('>', '')

df['content'] = df['content'].apply(remove_angle_brackets)


In [ ]:
# df.to_csv('fibromyalgia_forum_clean.csv', index=False, errors='ignore')

In [ ]:
# Recheck to confirm HTML tags have been removed
html_tag_post_clean = df['content'].apply(lambda x: bool(re.search(r'<[^>]+>', x))).sum()

html_tag_post_clean

np.int64(0)

In [ ]:
def clean_unicode_surrogates(x):
    if isinstance(x, str):
        return x.encode('utf-8', 'ignore').decode('utf-8', 'ignore')
    return x

# Apply this function to all cells in the DataFrame
df= df.applymap(clean_unicode_surrogates)


In [ ]:
df['content'].isnull().sum()

np.int64(0)

In [ ]:
df.to_csv('fibromyalgia_forum_cleaned.csv', index=False)


In [ ]:
df['content'].isnull().sum()

np.int64(0)

In [ ]:
missing_content_rows = df[df['content'].isnull()]
display(missing_content_rows)

,thread_id,thread_title,thread_url,forum_category,thread_views,thread_replies,total_posts,post_type,post_index,post_id,...,user_country,user_state,has_quotes,quote_count,all_quotes,quoted_users,timestamp_parsed,country_name,is_guest,user_tenure_days


In [ ]:
df = pd.read_csv('fibromyalgia_forum_cleaned.csv')
df['content'].isnull().sum()

np.int64(54)

In [ ]:
missing_content_rows = df[df['content'].isnull()]
display(missing_content_rows)

,thread_id,thread_title,thread_url,forum_category,thread_views,thread_replies,total_posts,post_type,post_index,post_id,...,user_country,user_state,has_quotes,quote_count,all_quotes,quoted_users,timestamp_parsed,country_name,is_guest,user_tenure_days
159,need-a-little-laugh-and-some-fun.30833,Need a little laugh and some fun,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,583,25,26,reply,1,post-329008,...,AU,NS,False,0,NaN,NaN,2025-04-26 09:42:19-04:00,Australia,False,1042.0
160,need-a-little-laugh-and-some-fun.30833,Need a little laugh and some fun,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,583,25,26,reply,2,post-329009,...,AU,NS,False,0,NaN,NaN,2025-04-26 09:52:11-04:00,Australia,False,1042.0
163,need-a-little-laugh-and-some-fun.30833,Need a little laugh and some fun,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,583,25,26,reply,5,post-329079,...,AU,NS,False,0,NaN,NaN,2025-04-29 10:05:02-04:00,Australia,False,1042.0
164,need-a-little-laugh-and-some-fun.30833,Need a little laugh and some fun,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,583,25,26,reply,6,post-329085,...,AU,NS,False,0,NaN,NaN,2025-04-29 21:23:06-04:00,Australia,False,1042.0
167,need-a-little-laugh-and-some-fun.30833,Need a little laugh and some fun,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,583,25,26,reply,9,post-329103,...,AU,NS,False,0,NaN,NaN,2025-05-01 10:40:15-04:00,Australia,False,1042.0
169,need-a-little-laugh-and-some-fun.30833,Need a little laugh and some fun,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,583,25,26,reply,11,post-329381,...,AU,WA,False,0,NaN,NaN,2025-05-17 18:56:08-04:00,Australia,False,996.0
174,need-a-little-laugh-and-some-fun.30833,Need a little laugh and some fun,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,583,25,26,reply,16,post-329387,...,AU,WA,False,0,NaN,NaN,2025-05-18 01:53:51-04:00,Australia,False,996.0
175,need-a-little-laugh-and-some-fun.30833,Need a little laugh and some fun,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,583,25,26,reply,17,post-329432,...,AU,WA,False,0,NaN,NaN,2025-05-20 00:14:05-04:00,Australia,False,996.0
176,need-a-little-laugh-and-some-fun.30833,Need a little laugh and some fun,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,583,25,26,reply,18,post-329496,...,AU,WA,False,0,NaN,NaN,2025-05-23 02:43:03-04:00,Australia,False,996.0
181,need-a-little-laugh-and-some-fun.30833,Need a little laugh and some fun,https://www.fibromyalgiaforums.org/community/t...,General discussions about fibromyalgia,583,25,26,reply,23,post-329854,...,AU,WA,False,0,NaN,NaN,2025-06-04 19:24:24-04:00,Australia,False,996.0


In [ ]:
# Drop rows where content is missing (critical for text analysis)
df = df.dropna(subset=['content'])

In [ ]:
missing_content_rows = df[df['content'].isnull()]
display(missing_content_rows)

,thread_id,thread_title,thread_url,forum_category,thread_views,thread_replies,total_posts,post_type,post_index,post_id,...,user_country,user_state,has_quotes,quote_count,all_quotes,quoted_users,timestamp_parsed,country_name,is_guest,user_tenure_days


In [ ]:
df.to_csv('fibromyalgia_forum_clean.csv', index=False)

In [ ]:
missing_content_rows = df[df['content'].isnull()]
display(missing_content_rows)

,thread_id,thread_title,thread_url,forum_category,thread_views,thread_replies,total_posts,post_type,post_index,post_id,...,user_country,user_state,has_quotes,quote_count,all_quotes,quoted_users,timestamp_parsed,country_name,is_guest,user_tenure_days
